In [ ]:
from xfact_lslms.client.lslms_client import LSMSClient
from tqdm import tqdm
import json
import random

In [ ]:
client_opt30b = LSMSClient(username='', password='', model_name='opt-iml-max-30b')
client_opt1b = LSMSClient(username='', password='', model_name='opt-iml-max-1.3b')
client_t0 = LSMSClient(username='', password='', model_name='t0pp')
client_t5 = LSMSClient(username='', password='', model_name='flan-t5-xxl')

generate_kwargs={'max_length': 100, 'penalty_alpha': 0.6, 'top_k': 4}
tokenizer_kwargs={'padding':True}

lines = open('GenRead/indatasets/nq/nq-test.jsonl', encoding='utf8').readlines()
lines = [json.loads(l) for l in lines]
num_samples = 4

prompt_s1 = "Generate a background document from Wikipedia to answer the given question. \n\n {query} \n\n"
prompt_s2 = "Refer to the passage below and answer the following question with just one entity. \n\n Passage: {background} \n\n Question: {query} \n\n The answer is"

In [ ]:
text = '''If 2 power 1 = 2, 2 power 2 = 4 then what is 2 power 9'''
#text = f"Reason with \n\n Question: {text} \n\n"

print(client_t5.call(text, generate_kwargs=generate_kwargs, tokenizer_kwargs=tokenizer_kwargs)['decoded_text'][0])
print()
print(client_t0.call(text, generate_kwargs=generate_kwargs, tokenizer_kwargs=tokenizer_kwargs)['decoded_text'][0])
print()
print(client_opt1b.call(text, generate_kwargs=generate_kwargs, tokenizer_kwargs=tokenizer_kwargs)['decoded_text'][0][len(text):])
print()
print(client_opt30b.call(text, generate_kwargs=generate_kwargs, tokenizer_kwargs=tokenizer_kwargs)['decoded_text'][0][len(text):])

In [ ]:
client_opt30b.call(text, generate_kwargs=generate_kwargs, tokenizer_kwargs=tokenizer_kwargs)

In [ ]:
for line in random.sample(lines, num_samples):
    question, answer = line['question'], line['answer']
    print(f'Question: {question} \t Answer: {answer}')
    
    query_s1 = prompt_s1.replace('{query}', question)
    query_s2 = prompt_s2.replace('{query}', question)
    
    ###########################################
    ###########################################

    responce = client_opt30b.call(query_s1, 
                           generate_kwargs=generate_kwargs,
                           tokenizer_kwargs=tokenizer_kwargs)

    background = responce['decoded_text'][0][len(query_s1):]
    print(f'Generated background using OPT_IML_MAX_30b is: {background}')

    responce = client_t5.call(query_s2.replace('{background}', background), 
                           generate_kwargs=generate_kwargs,
                           tokenizer_kwargs=tokenizer_kwargs)
    print(f"Repsonse: {responce['decoded_text']}")

    ###########################################
    responce = client_opt1b.call(query_s1, 
                           generate_kwargs=generate_kwargs,
                           tokenizer_kwargs=tokenizer_kwargs)

    background = responce['decoded_text'][0][len(query_s1):]
    print(f'Generated background using OPT_IML_MAX_1b is: {background}')

    responce = client_t5.call(query_s2.replace('{background}', background), 
                           generate_kwargs=generate_kwargs,
                           tokenizer_kwargs=tokenizer_kwargs)
    print(f"Repsonse: {responce['decoded_text']}")

    ###########################################
    responce = client_t0.call(query_s1, 
                           generate_kwargs=generate_kwargs,
                           tokenizer_kwargs=tokenizer_kwargs)

    background = responce['decoded_text'][0]
    print(f'Generated background using t0 is: {background}')

    responce = client_t0.call(query_s2.replace('{background}', background), 
                           generate_kwargs=generate_kwargs,
                           tokenizer_kwargs=tokenizer_kwargs)
    print(f"Repsonse: {responce['decoded_text']}")

    ###########################################
    responce = client_t5.call(query_s1, 
                           generate_kwargs=generate_kwargs,
                           tokenizer_kwargs=tokenizer_kwargs)

    background = responce['decoded_text'][0]
    print(f'Generated background using t5 is: {background}')

    responce = client_t5.call(query_s2.replace('{background}', background), 
                           generate_kwargs=generate_kwargs,
                           tokenizer_kwargs=tokenizer_kwargs)
    print(f"Repsonse: {responce['decoded_text']}")

    ###########################################
    
    
    print('\n\n')

In [ ]:
## Determanistic

# bf16:
# Propmt: What is the best movie by IMDb? Explain it's plot as well
# Responce: 'The movie is a great example of the movie. It is a great movie.'
# Prompt: translate English to German: The house is wonderful
# Responce: Das Haus ist schön.

# Float32
# Propmt: What is the best movie by IMDb? Explain it's plot as well
# Responce: The best movie by IMDb
# Prompt: translate English to German: The house is wonderful
# Responce: Das Haus ist schön.



In [ ]:
# Load an example from the datasets ag_news
from datasets import load_dataset
dataset = load_dataset("imdb", split="train")
example = dataset[1]

# Load prompts for this dataset
from promptsource.templates import DatasetTemplates
ag_news_prompts = DatasetTemplates('imdb')

# Print all the prompts available for this dataset. The keys of the dict are the uuids the uniquely identify each of the prompt, and the values are instances of `Template` which wraps prompts
print(ag_news_prompts.templates)




In [ ]:
ag_news_prompts

In [ ]:
# Select a prompt by its name
prompt = ag_news_prompts["classify_question_first"]

# Apply the prompt to the example
result = prompt.apply(example)
print("INPUT: ", result[0])
print("TARGET: ", result[1])